In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
!git clone https://github.com/ari-dasci/OD-WeaponDetection.git

Cloning into 'OD-WeaponDetection'...
remote: Enumerating objects: 50755, done.
remote: Counting objects: 100% (11673/11673), done.
remote: Compressing objects: 100% (10232/10232), done.
remote: Total 50755 (delta 11), reused 11673 (delta 11), pack-reused 39082
Receiving objects: 100% (50755/50755), 2.26 GiB | 18.69 MiB/s, done.
Resolving deltas: 100% (10978/10978), done.
Updating files: 100% (63128/63128), done.


In [ ]:
data_dir = './OD-WeaponDetection/Weapons and similar handled objects/Sohas_weapon-Classification'

In [ ]:
print(os.listdir(data_dir))

['pistol', 'knife', 'monedero', 'tarjeta', 'smartphone', 'billete']


In [ ]:
pistol_data = os.listdir(data_dir + '/pistol')

In [ ]:
print(len(os.listdir(data_dir)))

6


In [ ]:
print(len(pistol_data))

3975


In [ ]:
print(len(os.listdir(data_dir + '/')))

6


In [ ]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2, horizontal_flip=True)

In [ ]:
train_dataset = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

Found 7638 images belonging to 6 classes.


In [ ]:
val_dataset = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 1906 images belonging to 6 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

In [ ]:
test_dataset = test_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

Found 9544 images belonging to 6 classes.


In [ ]:
print(len(train_dataset))

239


In [ ]:
class WeaponClassificationBase(keras.Model):
  def training_step(self, batch):
    images, labels = batch
    out = self(images)
    loss = losses.binary_crossentropy(out, labels)
    return loss
  
  def validation_step(self, batch):
    images, labels = batch
    out = self(images)
    loss = losses.binary_crossentropy(out, labels)
    acc = accuracy(out, labels)
    return {'val_loss': loss.detach(), 'val_acc': acc}

  def validation_epoch_end(self, outputs):
    batch_losses = [x['val_loss'] for x in outputs]
    epoch_loss = tf.stack(batch_losses).mean()
    batch_accs = [x['val_acc'] for x in outputs]
    epoch_acc = tf.stack(batch_accs).mean()
    return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

  def epoch_end(self, epoch, result):
    print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['train_loss'], result['val_loss'], result['val_acc']))

def accuracy(outputs, labels):
    _, preds = tf.max(outputs, dim=1)
    return tf.tensor(tf.sum(preds == labels).item() / len(preds))

In [ ]:
class WeaponDetectionModel(WeaponClassificationBase):
  def __init__(self):
    super().__init__()
    self.network = models.Sequential(
        Conv2D(32, kernel_size = 3, padding = 'same'),
        layers.PReLU(),
        Conv2D(64, kernel_size = 3, strides = 1, padding = 'same'),
        layers.PReLU(),
        layers.MaxPool2D(pool_size=(2, 2)),

        Conv2D(128, kernel_size = 3, strides = 1, padding = 'same'),
        layers.PReLU(),
        Conv2D(128, kernel_size = 3, strides = 1, padding = 'same'),
        layers.PReLU(),
        layers.MaxPool2D(pool_size=(2, 2)),

        Conv2D(256, kernel_size = 3, strides = 1, padding = 'same'),
        layers.PReLU(),
        layers.MaxPool2D(pool_size=(2, 2)),

        Flatten(),
        layers.Dense(units=3, activation='softmax'))
    
  def forward(self, xb):
    return self.network.predict(xb)

In [ ]:
optimizer = optimizers.Adam(learning_rate=0.001)